# nb-model_xg-meta

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from collections import OrderedDict
from functools import partial, reduce
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fname = 'nb-model_xg-meta.ipynb'
dir_name = 'model'
fix_path(get_cwd(fname, dir_name +sep))

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from dask import delayed, compute
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import Dataset as TorchDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import torchfunc
from torchmeta.utils.data import BatchMetaDataLoader
import pytorch_lightning as pl

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 50)

from common_util import MODEL_DIR, RECON_DIR, JSON_SFX_LEN, DT_CAL_DAILY_FREQ, is_type, pd_common_idx_rows, remove_dups_list, NestedDefaultDict, set_loglevel, search_df, chained_filter, get_variants, load_df, dump_df, load_json, gb_transpose, pd_common_index_rows, filter_cols_below, inner_join, outer_join, ser_shift, list_get_dict, window_iter, benchmark
from common_util import isnt, window_iter, np_assert_identical_len_dim, midx_get_level, pd_rows, midx_intersect, pd_common_idx_rows, midx_split, pd_midx_to_arr, window_iter, np_at_least_nd, np_is_ndim, identity_fn
from model.common import DATASET_DIR, XG_PROCESS_DIR, XG_DATA_DIR, XG_DIR, PYTORCH_MODELS_DIR, ERROR_CODE, TEST_RATIO, VAL_RATIO, EXPECTED_NUM_HOURS, default_dataset
from model.common import PYTORCH_ACT_MAPPING, PYTORCH_OPT_MAPPING, PYTORCH_SCH_MAPPING, PYTORCH_LOSS_MAPPING
from model.xg_util import xgload
from model.preproc_util import temporal_preproc, stride_preproc
from model.train_util import pd_get_np_tvt, batchify
from model.pl_util import TCNModel
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

CRITICAL:root:script location: /home/kev/crunch/model/nb-model_xg-meta.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/


## Load Data

In [2]:
assets = ['sp_500', 'russell_2000', 'nasdaq_100', 'dow_jones']
chosen_asset = assets[0]

In [3]:
f = xgload(XG_DATA_DIR +'features' +sep)
l = xgload(XG_DATA_DIR +'labels' +sep)
t = xgload(XG_DATA_DIR +'targets' +sep)

In [4]:
print('num f: {}'.format(len(list(f))))
print('num l: {}'.format(len(list(l))))
print('num t: {}'.format(len(list(t))))

num f: 2520
num l: 1008
num t: 1504


### ddir / dret

In [5]:
ddir_pba_hoc = {a: list(l.childkeys([a, 'ddir', 'ddir', 'pba_hoc_hdxret_ddir'])) for a in assets}
ddir_vol_hoc = {a: list(l.childkeys([a, 'ddir', 'ddir', 'vol_hoc_hdxret_ddir'])) for a in assets}

In [6]:
dret_pba_hoc = {a: list(t.childkeys([a, 'dret', 'dret', 'pba_hoc_hdxret_dret'])) for a in assets}
dret_vol_hoc = {a: list(t.childkeys([a, 'dret', 'dret', 'vol_hoc_hdxret_dret'])) for a in assets}

### ddir1 / dret1

In [7]:
groups = ['lin', 'log']
fmt3, fmt4 = '{}_{}', '{}_hdxret1_{}'

In [8]:
e = 'ddir1'
b = 'pba_hoc'; ddir1_pba_hoc = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'pba_hlh'; ddir1_pba_hlh = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hoc'; ddir1_vol_hoc = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hlh'; ddir1_vol_hlh = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}

In [9]:
e = 'dret1'
b = 'pba_hoc'; dret1_pba_hoc = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'pba_hlh'; dret1_pba_hlh = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hoc'; dret1_vol_hoc = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hlh'; dret1_vol_hlh = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}

### ddir2/dret2

In [10]:
scalars = ['0.5', '1', '2']
stats = ['avg', 'std', 'mad', 'max', 'min']
fmt4, fmt5 = '{}_hdxret2_{}', '{}_hdxret2({}*{},1)_{}'

In [11]:
e = 'ddir2'
b = 'pba_hoc'; ddir2_pba_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'pba_hlh'; ddir2_pba_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hoc'; ddir2_vol_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hlh'; ddir2_vol_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}

In [12]:
e = 'dret2'
b = 'pba_hoc'; dret2_pba_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'pba_hlh'; dret2_pba_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hoc'; dret2_vol_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hlh'; dret2_vol_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}

### dxfbdir1 / dxfbret1

In [13]:
groups = ['lin', 'log']
fmt3, fmt4 = '{}_{}', '{}_hdxcret1_{}'

In [14]:
e = 'dxfbdir1'
b = 'pba_hoc'; dxfbdir1_pba_hoc = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'pba_hlh'; dxfbdir1_pba_hlh = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hoc'; dxfbdir1_vol_hoc = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hlh'; dxfbdir1_vol_hlh = {a: {g: list(l.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}

In [15]:
e = 'dxfbcret1'
#fmt3, fmt4 = '{}_{}', '{}_hdxcret1_{}'
b = 'pba_hoc'; dxfbcret1_pba_hoc = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'pba_hlh'; dxfbcret1_pba_hlh = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hoc'; dxfbcret1_vol_hoc = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hlh'; dxfbcret1_vol_hlh = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}

In [16]:
e = 'dxfbval1'
b = 'pba_hoc'; dxfbval1_pba_hoc = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'pba_hlh'; dxfbval1_pba_hlh = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hoc'; dxfbval1_vol_hoc = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}
b = 'vol_hlh'; dxfbval1_vol_hlh = {a: {g: list(t.childkeys([a, e, fmt3.format(e, g), fmt4.format(b, e)])) for g in groups} for a in assets}

### dxfbdir2 / dxfbcret2

In [17]:
scalars = ['0.5', '1', '2']
stats = ['avg', 'std', 'mad', 'max', 'min']
fmt4, fmt5 = '{}_hdxcret2_{}', '{}_hdxcret2({}*{},1)_{}'

In [18]:
e = 'dxfbdir2'
b = 'pba_hoc'; dxfbdir2_pba_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'pba_hlh'; dxfbdir2_pba_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hoc'; dxfbdir2_vol_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hlh'; dxfbdir2_vol_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}

In [19]:
e = 'dxfbcret2'
b = 'pba_hoc'; dxfbcret2_pba_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'pba_hlh'; dxfbcret2_pba_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hoc'; dxfbcret2_vol_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hlh'; dxfbcret2_vol_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}

In [20]:
e = 'dxfbval2'
b = 'pba_hoc'; dxfbval2_pba_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'pba_hlh'; dxfbval2_pba_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hoc'; dxfbval2_vol_hoc = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}
b = 'vol_hlh'; dxfbval2_vol_hlh = {a: {d: [[a, e, e, fmt4.format(b, e), fmt5.format(b, c, d, e)] for c in scalars] for d in stats} for a in assets}

### Chosen Labels/Targets

In [21]:
def get_lt(d, store, asset=chosen_asset, subset=None):
    """
    Return label or target data as a DataFrame.
    """
    kcs = d[asset][subset] if (subset) else d[asset]
    lt = pd.concat([store[kc] for kc in kcs], axis=1, keys=[kc[-1] for kc in kcs])
    lt.columns = lt.columns.droplevel(-1)
    return lt

#### PBA

In [44]:
ddir = get_lt(ddir_pba_hoc, l)
ddir1 = get_lt(ddir1_pba_hoc, l, subset='log')
dxfbdir1 = get_lt(dxfbdir1_pba_hoc, l, subset='log')

In [36]:
dret = get_lt(dret_pba_hoc, t)
dret1 = get_lt(dret1_pba_hoc, t, subset='log')
dxfbcret1 = get_lt(dxfbcret1_pba_hoc, t, subset='log')
dxfbval1 = get_lt(dxfbval1_pba_hoc, t, subset='log')

#### VOL

In [46]:
ddir_vol = get_lt(ddir_vol_hoc, l)
ddir1_vol = get_lt(ddir1_vol_hoc, l, subset='log')
dxfbdir1_vol = get_lt(dxfbdir1_vol_hoc, l, subset='log')

In [47]:
dret_vol = get_lt(dret_vol_hoc, t)
dret1_vol = get_lt(dret1_vol_hoc, t, subset='log')
dxfbcret1_vol = get_lt(dxfbcret1_vol_hoc, t, subset='log')
dxfbval1_vol = get_lt(dxfbval1_vol_hoc, t, subset='log')

### Features

In [24]:
list(set([k[1] for k in f.childkeys([assets[0]])]))

['hohlca',
 'dwrpt',
 'ddiff',
 'dwrmx',
 'dwrod',
 'dwrxmx',
 'hdgau',
 'hdod',
 'hduni',
 'dc',
 'hdmx',
 'hdpt',
 'hdzn',
 'dohlca',
 'dwrzn',
 'dlogret',
 'dffd']

### Chosen Features

In [20]:
kc_end = ['ddiff', 'ddiff_pba_vol']
ft_all = {a: list(f.childkeys([a, *kc_end])) for a in assets}
feat = ft_all[chosen_asset]

In [21]:
feat

[['sp_500',
  'ddiff',
  'ddiff_pba_vol',
  'pba_dohlca_ddiff',
  'pba_dohlca_ddiff(1)'],
 ['sp_500',
  'ddiff',
  'ddiff_pba_vol',
  'vol_dohlca_ddiff',
  'vol_dohlca_ddiff(1)']]

## Select Data

In [22]:
features_df = inner_join(f[feat[0]], f[feat[1]])
# Select non-zero values and apply to_bin (converts {-1, 0, 1} series to {0, 1} by setting -1->0), my code is stupid confusing
# Note that the label and target series are already pre-shifted from processing by xgp.py via the config files in  xg-process/
no_zero = lambda df: df[df.values.sum(axis=1) != 0]
to_bin = lambda df: (df+1)*.5
feature_df, label_df, target_df = pd_common_idx_rows(features_df, to_bin(no_zero(ddir)), dret)
assert(feature_df.shape[0]==label_df.shape[0]==target_df.shape[0])

## Meta Sampling

In [23]:
def __setup_data__(data):
    feature_df, label_df, target_df = data
    ftrain, fval, ftest = map(np_at_least_nd, pd_get_np_tvt(feature_df, as_midx=False))
    ltrain, lval, ltest = map(partial(np_at_least_nd, axis=-1), pd_get_np_tvt(label_df, as_midx=False))
    ttrain, tval, ttest = map(partial(np_at_least_nd, axis=-1), pd_get_np_tvt(target_df, as_midx=False))
    flt_train = (ftrain, ltrain, ttrain); np_assert_identical_len_dim(*flt_train)
    flt_val = (fval, lval, tval); np_assert_identical_len_dim(*flt_val)
    flt_test = (ftest, ltest, ttest); np_assert_identical_len_dim(*flt_test)

    #self.feat_shape = ftrain.shape # Required to infer input shape of model
    #self.num_unique_labels = max(map(lambda a: len(np.unique(a)), (ltrain, lval, ltest))) # TODO - set a variable for size of output layer automatically based on label_df
    return flt_train, flt_val, flt_test

In [24]:
def __preproc__(data, m_params):
    return temporal_preproc(data, window_size=m_params['window_size'], apply_idx=range(len(data)))

def __preproc2__(data, m_params):
    return stride_preproc(data, window_size=m_params['window_size'])

#@pl.data_loader
def train_dataloader(t_params, flt):
    logging.info('train_dataloader called')
    return batchify(t_params, __preproc__(flt), False)

#@pl.data_loader
def val_dataloader(t_params, flt):
    logging.info('val_dataloader called')
    return batchify(t_params, __preproc__(flt), False)

#@pl.data_loader
def test_dataloader(t_params, flt):
    logging.info('test_dataloader called')
    return batchify(t_params, __preproc__(flt), False)

In [25]:
train, val, test = __setup_data__((feature_df, label_df, target_df))

In [26]:
params = {
    'loss': 'nll',
    'batch_size': 1,
    'window_size': 20
}

In [27]:
def batchify(params, data, shuffle_batches=False):
	"""
	Return a torch.DataLoader made from a tuple of numpy arrays.

	Args:
		params (dict): model parameters dictionary
		data (tuple): tuple of numpy arrays, features are the first element
		shuffle_batches (bool): whether or not to shuffle the batches

	Returns:
		torch.DataLoader
	"""
	f = torch.tensor(data[0], dtype=torch.float32, requires_grad=True)
	if (params['loss'] in ('bce', 'bcel', 'mae', 'mse')):
		l = [torch.tensor(d, dtype=torch.float32, requires_grad=False) for d in data[1:]]
	elif (params['loss'] in ('ce', 'nll')):
		l = [torch.tensor(d, dtype=torch.int64, requires_grad=False).squeeze() for d in data[1:]]
	ds = TensorDataset(f, *l)
	dl = DataLoader(ds, batch_size=params['batch_size'], shuffle=shuffle_batches)
	return ds

In [28]:
params = {
    'loss': 'nll',
    'batch_size': 1,
    'window_size': 20
}

### Overlapping Episodes:

In [29]:
train_ol = __preproc__(train, params)
val_ol = __preproc__(val, params)
test_ol = __preproc__(test, params)
print(train_ol[0].shape, train_ol[1].shape)

(1337, 10, 20) (1337, 1, 20)


### Non-Overlapping Episodes:

In [30]:
train_nol = __preproc2__(train, params)
val_nol = __preproc2__(val, params)
test_nol = __preproc2__(test, params)
print(train_nol[0].shape, train_nol[1].shape)

(67, 10, 20) (67, 20, 1)


## Models

In [31]:
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, ElasticNet

In [32]:
tr_split = .75
w = params['window_size']
tr, ts = int(tr_split*w), (1-tr_split)*w
print(tr, ts)

scs = []
for x, y, z in zip(*train_nol):
    x, y = x.T, y
    clf = LogisticRegression(C=10**-2, l1_ratio=.9, penalty='elasticnet', random_state=0, solver='saga').fit(x[0:tr], y[0:tr].squeeze())
    #clf = LinearRegression().fit(x[0:tr], y[0:tr].squeeze())
    #yh = clf.predict(x[tr:])
    #|yp = clf.predict_proba(x[tr:])
    sc = clf.score(x[tr:], y[tr:].squeeze())
    scs.append(sc)
    #print(sc)
    #print(x)
    #print(y.squeeze())
    #print(z.T)

15 5.0


In [33]:
(np.array(scs).mean()-train_nol[1].squeeze().mean())*100

1.6417910447761197